In [20]:
# DeepLncLoc
from Bio import SeqIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import csv
options = Options()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"    #chrome binary location specified here
options.add_argument("--start-maximized") #open Browser in maximized mode
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
service = Service('/Users/chenkexin/Downloads/chromedriver-mac-arm64/chromedriver')
# driver = webdriver.Chrome(options=options, executable_path=r'D:\python39\Scripts\chromedriver.exe')
driver = webdriver.Chrome(options=options, service=service)
driver.get('https://www.digitalgeneai.tech/solution/affinity')



# 打开 TSV 文件并逐行读取
file_path = '/Users/chenkexin/Downloads/affinity (1)/final_dataset_train_no_du.tsv'

with open(file_path, 'r') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')  # 指定分隔符为制表符
    header = next(tsv_reader)  # 读取表头（如果有表头）
    print(f"表头: {header}")
    result=[]
    count=-1
    # 逐行读取内容
    for row in tsv_reader:# 最长等待 20 秒
        # 每行的元素存储在 `row` 列表中
        print(f"当前行: {row}")
        # 在这里对每行数据进行处理
        
        count+=1
        if count <= 38:
           continue
        print("现在进行第"+str(count)+"条序列...")
        '''
        driver.find_element(By.ID, 'Enter a heavy chain sequence').clear()
        driver.find_element(By.ID, 'Enter a heavy chain sequence').send_keys(">"+str(row[1]))
        driver.find_element(By.ID, 'Enter a light chain sequence').clear()
        driver.find_element(By.ID, 'Enter a light chain sequence').send_keys(">"+str(row[2]))
        driver.find_element(By.ID, 'Enter an antigen sequence').clear()
        driver.find_element(By.ID, 'Enter an antigen sequence').send_keys(">"+str(row[3]))
        '''
        # 等待页面加载完成
        WebDriverWait(driver, 5).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        
        # 定位并输入重链序列
        heavy_chain_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[1]/textarea'))
        )
        heavy_chain_input.clear()
        heavy_chain_input.send_keys(str(row[1]))
        
        # 定位并输入轻链序列
        light_chain_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[2]/textarea'))
        )
        light_chain_input.clear()
        light_chain_input.send_keys(str(row[2]))
        
        # 定位并输入抗原序列
        antigen_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[3]/textarea'))
        )
        antigen_input.clear()
        antigen_input.send_keys(str(row[3]))
        time.sleep(1)
        # submit
        driver.find_element(By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[2]/button[3]').click()
        time.sleep(10)
        # 找到预测结果//*[@id="app"]
        antigen_input = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[3]/span')))
        # 获取抗原序列结果的文本内容
        antigen_text = antigen_input.text  # 'Predicted binding affinity (∆G): -11.033 Kcal/mol'
        # 提取数字部分
        
        binding_affinity = re.search(r"-?\d+\.\d+", antigen_text).group()  # 提取 -11.033
        # 将结果存入列表
        res = [binding_affinity, row[0]]  # 创建一个包含结果和名称的列表
        result.append(res)
        if count % 10 == 0:
            df = pd.DataFrame(result, columns=["Binding Affinity", "name"])
            df.to_csv(f'./DGaffinity.csv')
print("爬取结束！保存结果...")
df=pd.DataFrame(columns=res_head,data=result)
df.to_csv('./DGaffinity.csv')
print(df)

表头: ['pdb_id', 'heavy_sequence', 'light_sequence', 'antigen_sequence', 'delta_g']
当前行: ['1AHW', 'DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKTLIYYATSLADGVPSRFSGSGSGQDYSLTISSLESDDTATYYCLQHGESPYTFGGGTKLEINRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC', 'EIQLQQSGAELVRPGALVKLSCKASGFNIKDYYMHWVKQRPEQGLEWIGLIDPENGNTIYDPKFQGKASITADTSSNTAYLQLSSLTSEDTAVYYCARDNSYYFDYWGQGTTLTVSSAKTTPPSVYPLAPGSAAQTNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSTWPSETVTCNVAHPASSTKVDKKI', 'TNTVAAYNLTWKSTNFKTILEWEPKPVNQVYTVQISTKSGDWKSKCFYTTDTECDLTDEIVKDVKQTYLARVFSYPAGNEPLYENSPEFTPYLETNLGQPTIQSFEQVGAAVNVTVEDERTLVRRNNTFLSLRDVFGKDLIYTLYYWKSSSSGKKTAKTNTNEFLIDVDKGENYCFSVQAVIPSRTVNRKSTDSPVECMG', '-10.9']
当前行: ['1AHW', 'DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKTLIYYATSLADGVPSRFSGSGSGQDYSLTISSLESDDTATYYCLQHGESPYTFGGGTKLEINRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC', 'EIQLQQSGAELVRPGALVKLSC

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=135.0.7049.96)
Stacktrace:
0   chromedriver                        0x00000001047f6e10 cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x00000001047ef0ac cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001043368d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x000000010431f96c chromedriver + 194924
4   chromedriver                        0x0000000104343338 cxxbridge1$string$len + 144836
5   chromedriver                        0x00000001043a5874 cxxbridge1$string$len + 547584
6   chromedriver                        0x00000001043be2c4 cxxbridge1$string$len + 648528
7   chromedriver                        0x0000000104371a80 cxxbridge1$string$len + 335116
8   chromedriver                        0x00000001047bbc98 cxxbridge1$str$ptr + 25750

In [21]:
df = pd.DataFrame(result, columns=["Binding Affinity", "name"])
df.to_csv(f'./DGaffinity.csv')

# baidu paddle dataset

In [59]:
# DeepLncLoc
from Bio import SeqIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import csv
options = Options()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"    #chrome binary location specified here
options.add_argument("--start-maximized") #open Browser in maximized mode
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
service = Service('/Users/chenkexin/Downloads/chromedriver-mac-arm64/chromedriver')
# driver = webdriver.Chrome(options=options, executable_path=r'D:\python39\Scripts\chromedriver.exe')
driver = webdriver.Chrome(options=options, service=service)
driver.get('https://www.digitalgeneai.tech/solution/affinity')



# 打开 TSV 文件并逐行读取
file_path = '/Users/chenkexin/Downloads/affinity (1)/final_dataset_train_no_du.tsv'

with open(file_path, 'r') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')  # 指定分隔符为制表符
    header = next(tsv_reader)  # 读取表头（如果有表头）
    print(f"表头: {header}")
    result=[]
    count=0
    # 逐行读取内容
    for row in tsv_reader:# 最长等待 20 秒
        # 每行的元素存储在 `row` 列表中
        #print(f"当前行: {row}")
        # 在这里对每行数据进行处理

        count+=1
        if count <= 2304:
            continue
        print("现在进行第"+str(count)+"条序列...")
        '''
        driver.find_element(By.ID, 'Enter a heavy chain sequence').clear()
        driver.find_element(By.ID, 'Enter a heavy chain sequence').send_keys(">"+str(row[1]))
        driver.find_element(By.ID, 'Enter a light chain sequence').clear()
        driver.find_element(By.ID, 'Enter a light chain sequence').send_keys(">"+str(row[2]))
        driver.find_element(By.ID, 'Enter an antigen sequence').clear()
        driver.find_element(By.ID, 'Enter an antigen sequence').send_keys(">"+str(row[3]))
        '''
        # 等待页面加载完成
        WebDriverWait(driver, 5).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )

        # 定位并输入重链序列
        heavy_chain_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[1]/textarea'))
        )
        heavy_chain_input.clear()
        heavy_chain_input.send_keys(str(row[1]).replace("\n", "").replace("\r", "").replace(" ", ""))

        # 定位并输入轻链序列
        light_chain_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[2]/textarea'))
        )
        light_chain_input.clear()
        light_chain_input.send_keys(str(row[2]).replace("\n", "").replace("\r", "").replace(" ", ""))

        # 定位并输入抗原序列
        antigen_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[3]/textarea'))
        )
        antigen_input.clear()
        antigen_input.send_keys(str(row[3]).replace("\n", "").replace("\r", "").replace(" ", ""))
        time.sleep(1)
        # submit
        driver.find_element(By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[2]/button[3]').click()
        time.sleep(10)
        # 找到预测结果//*[@id="app"]
        antigen_input = WebDriverWait(driver, 3000).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[3]/span')))
        # 获取抗原序列结果的文本内容
        antigen_text = antigen_input.text  # 'Predicted binding affinity (∆G): -11.033 Kcal/mol'
        # 提取数字部分
        print(antigen_text)
        binding_affinity = re.search(r"-?\d+\.\d+", antigen_text).group()  # 提取 -11.033
        # 将结果存入列表
        res = [binding_affinity, row[0]]  # 创建一个包含结果和名称的列表
        result.append(res)
        if count % 10 == 0:
            df = pd.DataFrame(result, columns=["Binding Affinity", "name"])
            df.to_csv(f'./DGaffinity.csv')
print("爬取结束！保存结果...")
df=pd.DataFrame(columns=res_head,data=result)
df.to_csv('./DGaffinity.csv')
print(df)

表头: ['pdb_id', 'heavy_sequence', 'light_sequence', 'antigen_sequence', 'delta_g']
现在进行第2305条序列...
Predicted binding affinity (∆G): -12.386 Kcal/mol
现在进行第2306条序列...
Predicted binding affinity (∆G): -12.386 Kcal/mol
现在进行第2307条序列...
Predicted binding affinity (∆G): -12.386 Kcal/mol
现在进行第2308条序列...
Predicted binding affinity (∆G): -10.448 Kcal/mol
现在进行第2309条序列...
Predicted binding affinity (∆G): -10.448 Kcal/mol
现在进行第2310条序列...
Predicted binding affinity (∆G): -10.448 Kcal/mol
现在进行第2311条序列...
Predicted binding affinity (∆G): -9.557 Kcal/mol
现在进行第2312条序列...
Predicted binding affinity (∆G): -9.557 Kcal/mol
现在进行第2313条序列...
Predicted binding affinity (∆G): -9.05 Kcal/mol
现在进行第2314条序列...
Predicted binding affinity (∆G): -9.05 Kcal/mol
现在进行第2315条序列...
Predicted binding affinity (∆G): -12.993 Kcal/mol
现在进行第2316条序列...
Predicted binding affinity (∆G): -12.993 Kcal/mol
现在进行第2317条序列...
Predicted binding affinity (∆G): -12.993 Kcal/mol
现在进行第2318条序列...
Predicted binding affinity (∆G): -9.558 Kcal/mol
现

NameError: name 'res_head' is not defined

In [60]:
df = pd.DataFrame(result, columns=["Binding Affinity", "name"])
df.to_csv(f'./DGaffinity.csv')

In [71]:
# DeepLncLoc
from Bio import SeqIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import csv
options = Options()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"    #chrome binary location specified here
options.add_argument("--start-maximized") #open Browser in maximized mode
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
service = Service('/Users/chenkexin/Downloads/chromedriver-mac-arm64/chromedriver')
# driver = webdriver.Chrome(options=options, executable_path=r'D:\python39\Scripts\chromedriver.exe')
driver = webdriver.Chrome(options=options, service=service)
driver.get('https://www.digitalgeneai.tech/solution/affinity')



# 打开 TSV 文件并逐行读取
file_path = '/Users/chenkexin/Downloads/affinity (1)/abbind_dataset_filtered.tsv'

with open(file_path, 'r') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')  # 指定分隔符为制表符
    header = next(tsv_reader)  # 读取表头（如果有表头）
    print(f"表头: {header}")
    result=[]
    count=0
    # 逐行读取内容
    for row in tsv_reader:# 最长等待 20 秒
        # 每行的元素存储在 `row` 列表中
        #print(f"当前行: {row}")
        # 在这里对每行数据进行处理

        count+=1

        if count <= 560:
            continue

        print("现在进行第"+str(count)+"条序列...")
        '''
        driver.find_element(By.ID, 'Enter a heavy chain sequence').clear()
        driver.find_element(By.ID, 'Enter a heavy chain sequence').send_keys(">"+str(row[1]))
        driver.find_element(By.ID, 'Enter a light chain sequence').clear()
        driver.find_element(By.ID, 'Enter a light chain sequence').send_keys(">"+str(row[2]))
        driver.find_element(By.ID, 'Enter an antigen sequence').clear()
        driver.find_element(By.ID, 'Enter an antigen sequence').send_keys(">"+str(row[3]))
        '''
        # 等待页面加载完成
        WebDriverWait(driver, 5).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )

        # 定位并输入重链序列
        heavy_chain_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[1]/textarea'))
        )
        heavy_chain_input.clear()
        heavy_chain_input.send_keys(str(row[1]).replace("\n", "").replace("\r", "").replace(" ", ""))

        # 定位并输入轻链序列
        light_chain_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[2]/textarea'))
        )
        light_chain_input.clear()
        light_chain_input.send_keys(str(row[2]).replace("\n", "").replace("\r", "").replace(" ", ""))

        # 定位并输入抗原序列
        antigen_input = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[1]/div[3]/textarea'))
        )
        antigen_input.clear()
        antigen_input.send_keys(str(row[3]).replace("\n", "").replace("\r", "").replace(" ", ""))
        time.sleep(1)
        # submit
        driver.find_element(By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[2]/button[3]').click()
        time.sleep(10)
        # 找到预测结果//*[@id="app"]
        antigen_input = WebDriverWait(driver, 3000).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div/div/div[2]/div[3]/span')))
        # 获取抗原序列结果的文本内容
        antigen_text = antigen_input.text  # 'Predicted binding affinity (∆G): -11.033 Kcal/mol'
        # 提取数字部分
        print(antigen_text)
        binding_affinity = re.search(r"-?\d+\.\d+", antigen_text).group()  # 提取 -11.033
        # 将结果存入列表
        res = [binding_affinity, row[0]]  # 创建一个包含结果和名称的列表
        result.append(res)
        if count % 10 == 0:
            df = pd.DataFrame(result, columns=["Binding Affinity", "name"])
            df.to_csv(f'./DGaffinity_abbind_dataset.csv')
print("爬取结束！保存结果...")
df=pd.DataFrame(columns=res_head,data=result)
df.to_csv('./DGaffinity_abbind_dataset.csv')
print(df)

表头: ['pdb_id', 'heavy_sequence', 'light_sequence', 'antigen_sequence', 'delta_g']
现在进行第561条序列...
Predicted binding affinity (∆G): -13.226 Kcal/mol
现在进行第562条序列...
Predicted binding affinity (∆G): -13.226 Kcal/mol
现在进行第563条序列...
Predicted binding affinity (∆G): -13.226 Kcal/mol
现在进行第564条序列...
Predicted binding affinity (∆G): -9.341 Kcal/mol
现在进行第565条序列...
Predicted binding affinity (∆G): -9.341 Kcal/mol
现在进行第566条序列...
Predicted binding affinity (∆G): -12.387 Kcal/mol
现在进行第567条序列...
Predicted binding affinity (∆G): -12.387 Kcal/mol
现在进行第568条序列...
Predicted binding affinity (∆G): -9.71 Kcal/mol
现在进行第569条序列...
Predicted binding affinity (∆G): -9.71 Kcal/mol
现在进行第570条序列...
Predicted binding affinity (∆G): -9.71 Kcal/mol
现在进行第571条序列...
Predicted binding affinity (∆G): -9.444 Kcal/mol
现在进行第572条序列...
Predicted binding affinity (∆G): -9.444 Kcal/mol
现在进行第573条序列...
Predicted binding affinity (∆G): -8.968 Kcal/mol
现在进行第574条序列...
Predicted binding affinity (∆G): -8.968 Kcal/mol
现在进行第575条序列...
Predi

NameError: name 'res_head' is not defined

In [73]:
df = pd.DataFrame(result, columns=["Binding Affinity", "name"])
df.to_csv(f'./DGaffinity_abbind_dataset.csv')